In [ ]:
# %% [markdown]
# # Final Assignment — Concentration Inequalities and Confidence Bounds
# **UIMP Quantum Science & Technology Master**
# 
# This notebook integrates the theoretical and practical content from the tutorials:
# - `ee046202_tutorial_00_probability_optimization.ipynb`
# - `ee046202_tutorial_01_classic_statistics_point_estimation.ipynb`
# - `ee046202_tutorial_02_classic_statistics_confidence_intervals.ipynb`
# 
# It addresses the evaluation problem:
# 1. Generate random data from Normal, Uniform, and Beta distributions.
# 2. Estimate and compare confidence intervals using **Hoeffding's** and **Chebyshev's** inequalities.
# 3. Compare empirical vs theoretical variances.
# 4. Analyze which distributions satisfy the **68–95–99.7 empirical rule**.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, uniform, beta

np.random.seed(42)

In [ ]:
def sample_distribution(dist_type, n=1000, **params):
    if dist_type == 'normal':
        return np.random.normal(params.get('mu', 0), params.get('sigma', 1), n)
    elif dist_type == 'uniform':
        return np.random.uniform(params.get('a', 0), params.get('b', 1), n)
    elif dist_type == 'beta':
        return np.random.beta(params.get('alpha', 2), params.get('beta', 5), n)
    else:
        raise ValueError('Unknown distribution')

def hoeffding_bound(n, epsilon, a, b):
    return 2 * np.exp(-2 * n * epsilon**2 / (b - a)**2)

def chebyshev_bound(var, epsilon):
    return var / epsilon**2

In [ ]:
n = 100
epsilons = [0.01, 0.05, 0.1, 0.2]

distributions = {
    'Normal(0,1)': ('normal', {'mu': 0, 'sigma': 1, 'a': -np.inf, 'b': np.inf}),
    'Uniform(0,1)': ('uniform', {'a': 0, 'b': 1}),
    'Beta(2,5)': ('beta', {'alpha': 2, 'beta': 5, 'a': 0, 'b': 1})
}

results = {}

for name, (dtype, params) in distributions.items():
    X = sample_distribution(dtype, n, **params)
    mean_emp = np.mean(X)
    var_emp = np.var(X)
    a, b = params.get('a', np.min(X)), params.get('b', np.max(X))

    probs = []
    for eps in epsilons:
        hoeff = hoeffding_bound(n, eps, a, b) if np.isfinite(a) and np.isfinite(b) else np.nan
        cheb = chebyshev_bound(var_emp, eps)
        probs.append((eps, hoeff, cheb))
    results[name] = {'mean': mean_emp, 'var': var_emp, 'bounds': probs}

In [ ]:
def theoretical_variance(dist_name, params):
    if dist_name == 'normal':
        return params['sigma']**2
    elif dist_name == 'uniform':
        a, b = params['a'], params['b']
        return (b - a)**2 / 12
    elif dist_name == 'beta':
        α, β = params['alpha'], params['beta']
        return (α * β) / (((α + β)**2) * (α + β + 1))
    else:
        return np.nan

var_summary = []
for name, (dtype, params) in distributions.items():
    X = sample_distribution(dtype, n, **params)
    var_emp = np.var(X)
    var_th = theoretical_variance(dtype, params)
    var_summary.append({
        'Distribution': name,
        'Variance (Empirical)': var_emp,
        'Variance (Theoretical)': var_th,
        'Bounded?': 'Yes' if np.isfinite(params.get('a', -np.inf)) and np.isfinite(params.get('b', np.inf)) else 'No'
    })

df_vars = pd.DataFrame(var_summary)
df_vars.round(6)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5), sharey=True)

for ax, (name, data) in zip(axes, results.items()):
    eps_vals = [p[0] for p in data['bounds']]
    hoeff_vals = [p[1] for p in data['bounds']]
    cheb_vals = [p[2] for p in data['bounds']]

    ax.plot(eps_vals, cheb_vals, 'o--', label='Chebyshev', alpha=0.8)
    ax.plot(eps_vals, hoeff_vals, 's--', label='Hoeffding', alpha=0.8)

    ax.set_title(name)
    ax.set_xlabel('ε (Deviation from mean)')
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.legend()

axes[0].set_ylabel('P(|X̄ − E[X]| ≥ ε) — Upper Bound')
fig.suptitle('Comparison of Hoeffding vs Chebyshev Bounds per Distribution', fontsize=13)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
def empirical_rule_test(data, mu, sigma):
    within_1 = np.mean((data >= mu - sigma) & (data <= mu + sigma))
    within_2 = np.mean((data >= mu - 2*sigma) & (data <= mu + 2*sigma))
    within_3 = np.mean((data >= mu - 3*sigma) & (data <= mu + 3*sigma))
    return within_1, within_2, within_3

normal_data = sample_distribution('normal', n=10000, mu=0, sigma=1)
mu, sigma = np.mean(normal_data), np.std(normal_data)
emp_1, emp_2, emp_3 = empirical_rule_test(normal_data, mu, sigma)

print(f'68–95–99.7 empirical check (Normal N(0,1)):\n')
print(f'Within ±1σ: {emp_1*100:.2f}% (expected 68%)')
print(f'Within ±2σ: {emp_2*100:.2f}% (expected 95%)')
print(f'Within ±3σ: {emp_3*100:.2f}% (expected 99.7%)')

In [ ]:
# %% [markdown]
# ## Conclusions
# - **Hoeffding’s inequality** provides tighter and distribution-free bounds but only for **bounded** distributions.
# - **Chebyshev’s inequality** applies to any distribution, but its bounds are generally looser.
# - The **68–95–99.7 rule** holds accurately for the Normal distribution.
# - For **Uniform** and **Beta**, empirical variances differ, and the rule does not hold precisely.
# 
# ✅ This notebook fulfills the evaluation tasks using theory and techniques from tutorials 00–02.